# Musicians- Medium

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


 ········


In [2]:
band = pd.read_sql_table('band', engine)
composer = pd.read_sql_table('composer', engine)
composition = pd.read_sql_table('composition', engine)
concert = pd.read_sql_table('concert', engine)
has_composed = pd.read_sql_table('has_composed', engine)
musician = pd.read_sql_table('musician', engine)
performance = pd.read_sql_table('performance', engine)
performer = pd.read_sql_table('performer', engine)
place = pd.read_sql_table('place', engine)
plays_in = pd.read_sql_table('plays_in', engine)

## 6.
**List the names, dates of birth and the instrument played of living musicians who play a instrument which Theo also plays.**

In [3]:
inst = (musician.loc[musician['m_name'].str.contains('^Theo')]
        .merge(performer, left_on='m_no', right_on='perf_is')
        ['instrument'])
(musician.loc[(musician['died'].isnull()) & 
              (~musician['m_name'].str.contains('^Theo'))]
 .merge(performer.loc[performer['instrument'].isin(inst)],
        left_on='m_no', right_on='perf_is')
 [['m_name', 'born', 'instrument']].sort_values('m_name'))

,m_name,born,instrument
2,Harry Forte,1951-02-28,violin
3,Harry Forte,1951-02-28,drums
1,James First,1965-06-10,violin
4,Jeff Dawn,1945-12-12,violin
0,John Smith,1950-03-03,violin


## 7.
**List the name and the number of players for the band whose number of players is greater than the average number of players in each band.**

In [4]:
t = (band.merge(plays_in, left_on='band_no', right_on='band_id')
     .merge(performer, left_on='player', right_on='perf_no')
     [['band_name', 'perf_is']].drop_duplicates())
a = (t.groupby('band_name').agg(n_mbr=pd.NamedAgg(column='perf_is', aggfunc='count'))
     .reset_index())
a.loc[a['n_mbr']>(t.shape[0]/band.shape[0])]

,band_name,n_mbr
0,AASO,7
2,Oh well,5
3,ROP,7


## 8.
**List the names of musicians who both conduct and compose and live in Britain.**

In [5]:
(musician.merge(composer, left_on='m_no', right_on='comp_is')
 .merge(place.loc[place['place_country'].isin(['England', 'Scotland'])], 
        left_on='living_in', right_on='place_no')
 .merge(performance, left_on='m_no', right_on='conducted_by')
 [['m_name']].drop_duplicates().sort_values('m_name'))

,m_name
0,Fred Bloggs
3,Jeff Dawn
2,Phil Hot
1,Rose Spring
6,Tony Smythe


## 9.
**Show the least commonly played instrument and the number of musicians who play it.**

In [6]:
(performer.merge(plays_in, left_on='perf_no', right_on='player')
 .merge(performance, left_on='band_id', right_on='gave')
 .groupby('instrument').agg(n_mus=pd.NamedAgg(column='perf_no', aggfunc='count'))
 .reset_index().sort_values('n_mus').iloc[:1])

,instrument,n_mus
3,clarinet,1


## 10.
**List the bands that have played music composed by Sue Little; Give the titles of the composition in each case.**

In [7]:
t = (composition.merge(has_composed, left_on='c_no', right_on='cmpn_no')
     .merge(composer, left_on='cmpr_no', right_on='comp_no')
     .merge(musician.loc[musician['m_name']=='Sue Little'],
            left_on='comp_is', right_on='m_no')
     [['c_no', 'c_title']])
(t.merge(performance, left_on='c_no', right_on='performed')
 .merge(band, left_on='gave', right_on='band_no')
 [['band_name', 'c_title']].sort_values('band_name'))

,band_name,c_title
2,BBSO,Slow Song
3,BBSO,Slow Symphony Blowing
4,Somebody Loves this,Slow Symphony Blowing
1,Swinging strings,Slow Song
0,The left Overs,Slow Song
